<a href="https://colab.research.google.com/github/Divyan7/mini-project-on-pandas/blob/main/lab7_AP23110010152.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install gdown
!gdown 1ZdhRqYv-JizWV6DxO6C4R_k1kxPhmlF2 -O dataset.csv



Downloading...
From: https://drive.google.com/uc?id=1ZdhRqYv-JizWV6DxO6C4R_k1kxPhmlF2
To: /content/dataset.csv
100% 14.6k/14.6k [00:00<00:00, 37.5MB/s]


In [7]:
import pandas as pd

df = pd.read_csv('dataset.csv')
display(df.head())

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,class
0,3,12669,9656,7561,214,2674,1338,2
1,3,7057,9810,9568,1762,3293,1776,2
2,3,6353,8808,7684,2405,3516,7844,2
3,3,13265,1196,4221,6404,507,1788,1
4,3,22615,5410,7198,3915,1777,5185,1


'Original dataset shape:'

(440, 9)

'Resampled dataset shape:'

(540, 8)

In [8]:
display(df.isnull().sum())

,0
Region,0
Fresh,0
Milk,0
Grocery,0
Frozen,0
Detergents_Paper,0
Delicassen,0
class,0


In [13]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Region            440 non-null    int64  
 1   Fresh             440 non-null    int64  
 2   Milk              440 non-null    int64  
 3   Grocery           440 non-null    int64  
 4   Frozen            440 non-null    int64  
 5   Detergents_Paper  440 non-null    int64  
 6   Delicassen        440 non-null    int64  
 7   class             440 non-null    int64  
 8   Region_encoded    440 non-null    float64
dtypes: float64(1), int64(8)
memory usage: 31.1 KB


None

In [14]:
from sklearn.preprocessing import StandardScaler

# Identify numerical columns to standardize (excluding the target and encoded columns if applicable)
numerical_cols = ['Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen']

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit and transform the numerical columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Display the head of the standardized DataFrame
display(df.head())

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,class,Region_encoded
0,3,0.052933,0.523568,-0.041115,-0.589367,-0.043569,-0.066339,2,12533.471519
1,3,-0.391302,0.544458,0.170318,-0.270136,0.086407,0.089151,2,12533.471519
2,3,-0.447029,0.408538,-0.028157,-0.137536,0.133232,2.243293,2,12533.471519
3,3,0.100111,-0.624020,-0.392977,0.687144,-0.498588,0.093411,1,12533.471519
4,3,0.840239,-0.052396,-0.079356,0.173859,-0.231918,1.299347,1,12533.471519


In [15]:
display(df['class'].value_counts())

,count
class,
2,180
3,173
1,87


In [19]:
from imblearn.over_sampling import SMOTE

# Separate features (X) and target (y)
X = df.drop(['class', 'Region_encoded'], axis=1)
y = df['class']

# Create a SMOTE instance
smote = SMOTE(random_state=42)

# Apply SMOTE to the data
X_resampled, y_resampled = smote.fit_resample(X, y)

# Display the new class distribution
display(y_resampled.value_counts())

,count
class,
2,180
1,180
3,180


In [20]:
display(X_resampled.var())

,0
Region,0.597279
Fresh,0.872072
Milk,0.857451
Grocery,0.861009
Frozen,0.904352
Detergents_Paper,0.852811
Delicassen,0.833363


In [21]:
display(X_resampled.corr(method='pearson'))

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
Region,1.000000,0.040087,0.043011,0.014481,0.016354,0.002605,0.040464
Fresh,0.040087,1.000000,0.069023,-0.040210,0.336842,-0.127652,0.227224
Milk,0.043011,0.069023,1.000000,0.738351,0.114011,0.671762,0.410616
Grocery,0.014481,-0.040210,0.738351,1.000000,-0.047913,0.925822,0.217305
Frozen,0.016354,0.336842,0.114011,-0.047913,1.000000,-0.139935,0.370100
Detergents_Paper,0.002605,-0.127652,0.671762,0.925822,-0.139935,1.000000,0.082985
Delicassen,0.040464,0.227224,0.410616,0.217305,0.370100,0.082985,1.000000


In [22]:
correlation_matrix = X_resampled.corr(method='pearson')

# Find features with absolute correlation above 0.5 (excluding self-correlation)
high_correlation_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.5:
            colname_i = correlation_matrix.columns[i]
            colname_j = correlation_matrix.columns[j]
            high_correlation_pairs.append((colname_i, colname_j, correlation_matrix.iloc[i, j]))

if high_correlation_pairs:
    display("Features with Pearson correlation above 0.5 (absolute value):")
    for feature1, feature2, correlation in high_correlation_pairs:
        display(f"- {feature1} and {feature2}: {correlation:.4f}")
else:
    display("No features found with Pearson correlation above 0.5 (absolute value).")

'Features with Pearson correlation above 0.5 (absolute value):'

'- Grocery and Milk: 0.7384'

'- Detergents_Paper and Milk: 0.6718'

'- Detergents_Paper and Grocery: 0.9258'

In [23]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets
display("Shape of X_train:", X_train.shape)
display("Shape of X_test:", X_test.shape)
display("Shape of y_train:", y_train.shape)
display("Shape of y_test:", y_test.shape)

'Shape of X_train:'

(432, 7)

'Shape of X_test:'

(108, 7)

'Shape of y_train:'

(432,)

'Shape of y_test:'

(108,)

In [26]:
def knn(X, y, queryPoint, k):
    """predict the class label for the query point"""
    # Euclidean Distance
    dist = np.sqrt(np.sum((queryPoint - X) ** 2, axis=1))

    # Storing distance and class labels together
    distances = [(dist[i], y[i]) for i in range(len(dist))]
    # sort the distances
    distances = sorted(distances)
    # Nearest/first k points
    distances = distances[:k]

    distances = np.array(distances)

    classes_counts = np.unique(distances[:,1], return_counts=True)

    index = classes_counts[1].argmax()
    pred = classes_counts[0][index]

    return int(pred), distances

In [33]:
# Assuming knn function is defined in a previous cell
import numpy as np

# Select a query point from the test set
query_point = X_test.iloc[50]
k_value = 5

# Use the knn function defined in a previous cell
pred, neighbors = knn(X_resampled, y_resampled, query_point, k_value)

print(f'k nearest neighbors with the distance and class label : {neighbors}')

print(f'The predicted class label: {pred}')

k nearest neighbors with the distance and class label : [[0.         3.        ]
 [0.29301359 3.        ]
 [0.54954236 3.        ]
 [0.60412883 3.        ]
 [0.63857556 3.        ]]
The predicted class label: 3


In [34]:
# Assuming knn function is defined in a previous cell
import numpy as np

# Select the same query point from the test set
query_point = X_test.iloc[50]
k_value = 7 # Set k value to 7

# Use the knn function defined in a previous cell
pred, neighbors = knn(X_resampled, y_resampled, query_point, k_value)

print(f'k nearest neighbors with the distance and class label : {neighbors}')

print(f'The predicted class label: {pred}')

k nearest neighbors with the distance and class label : [[0.         3.        ]
 [0.29301359 3.        ]
 [0.54954236 3.        ]
 [0.60412883 3.        ]
 [0.63857556 3.        ]
 [0.65972626 3.        ]
 [0.67278063 3.        ]]
The predicted class label: 3
